<a href="https://colab.research.google.com/github/yeeeeeun/Jeju_dialect_translation/blob/main/Jeju_dialect_translation_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
!pip install accelerate>=0.20.1

In [ ]:
!pip install transformers

In [ ]:
batch_size = 32
learning_rate = 0.001
epochs = 10

args = {
    "learning_rate": learning_rate,
    "epochs": epochs,
    "batch_size": batch_size
}

# batch_size, lr, epochs를 미리 선언해주었습니다. (학습 관련 설정- 하이퍼파라미터)

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import PreTrainedTokenizerFast, BartModel,BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2') # koBART 모델의 토크나이저 사용

model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v2') # 사전 학습된 koBART 모델 로드

device = torch.device("cuda:0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/495M [00:00<?, ?B/s]

In [ ]:
model = model.to(device)

In [ ]:
pip install pandas openpyxl xlrd

In [ ]:
import pandas as pd

excel_file_path = '/content/drive/MyDrive/NLP/big/datasets_final.xlsx'

data = pd.read_excel(excel_file_path, sheet_name='train')

print(data.head())

  standard_form dialect_form
0      너 막 지쳤네?     너 막 지쳤져?
1     막 지쳤네 얼굴이   막 지쳐신게 얼굴이
2     성당 갔다오고 예    성당 갔다오고 예
3           성당?          성당?
4        그래서 이제      겅해그네 이제


In [ ]:
data.tail()

,standard_form,dialect_form
2206,그니까 우리 언제 아이들을 데리고 밥이나 먹자,게난 우리 언제 아이들을 데령 밥이나 먹게
2207,바쁜 아이들 해서,바쁜 아이들 해그네
2208,밥이나 한번 먹게,밥이나 한번 먹게
2209,그렇게 해요,겅 허게마씸
2210,알겠어 끝,알아서 끝


In [ ]:

max_length = data.iloc[:, 0].str.len().max()


longest_sentence = data.loc[data.iloc[:, 0].str.len() == max_length, data.columns[0]].values[0]

print("가장 긴 문장의 글자수:", max_length)
print("가장 긴 문장을 출력함:", longest_sentence)

가장 긴 문장의 글자수: 52
가장 긴 문장을 출력함: 일 학년 이 학기때부터 막 그 친구들끼리 이렇게하고 막 그렇게 하더라고요 이 학년 일 학기 때


## 데이터 로드 및 전처리 진행

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

x = data['standard_form']
y = data['dialect_form']

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42)

print("Train 데이터 크기:", x_train.shape[0])
print("Valid 데이터 크기:", x_valid.shape[0])


Train 데이터 크기: 1768
Valid 데이터 크기: 443


## Dataset 클래스 정의

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence


class dialectDataset(Dataset):
    def __init__(self, standard, dialect, tokenizer):
        self.standard = standard
        self.dialect = dialect
        self.tokenizer = tokenizer
    def __len__(self):
        return len(self.standard)
    def __getitem__(self, idx):
        standard_text = self.standard[idx]
        dialect_text = self.dialect[idx]


        inputs = self.tokenizer(
            text=dialect_text,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            max_length=160,
            )

        outputs = self.tokenizer(
            text=standard_text,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            max_length=130,
            )

        sample = {
            'input_ids': inputs.input_ids.squeeze(),
            'attention_mask': inputs.attention_mask.squeeze(),
            'labels': outputs.input_ids.squeeze()
        }

        return sample

train_dataset = dialectDataset(x_train.values, y_train.values,tokenizer)
valid_dataset = dialectDataset(x_valid.values, y_valid.values,tokenizer)

In [ ]:
from transformers import TrainingArguments
from transformers import Trainer
from transformers import DataCollatorWithPadding
from transformers import EvalPrediction

def compute_metrics(p: EvalPrediction):
    preds = p.predictions
    labels = p.label_ids

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    references = [[label.strip()] for label in decoded_labels]
    predictions = [pred.strip() for pred in decoded_preds]

    from nltk.translate.bleu_score import corpus_bleu
    bleu_score = corpus_bleu(references, predictions)

    return {"bleu_score": bleu_score}


training_args = TrainingArguments(
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    output_dir="/content/drive/MyDrive/NLP/pretrained",
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/NLP/pretrained",
    logging_steps=10,
    num_train_epochs=10,
    save_total_limit=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics

)


In [ ]:

trainer.train()

trainer.save_model('')


Step,Training Loss
10,18.577900
20,16.295700
30,12.239000
40,9.266200
50,7.300100
60,5.886300
70,5.080100
80,4.569200
90,4.115800
100,3.702000


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}
